<a href="https://colab.research.google.com/github/pannettonne/Portfolio/blob/main/LaSalle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import importlib.util
import subprocess
#import pkg_resources
import sys
print("Empiezo")

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import os
import yagmail
from premailer import Premailer

#import pyjuice as juice
import css_inline

import re
import unicodedata
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta

def minify_html(html: str) -> str:
    html = re.sub(r'>\s+<', '><', html)  # elimina espacios/saltos entre etiquetas
    html = re.sub(r'\s{2,}', ' ', html)  # colapsa espacios repetidos
    return html.strip()

def norm_text(s: str) -> str:
    s = unicodedata.normalize("NFKC", s).replace("\xa0", " ")
    s = s.replace("–", "-").replace("—", "-").replace("\u2011","-").replace("\u200b","")
    return " ".join(s.split())

os.environ['KEYRING_BACKEND'] = 'keyrings.alt.chainer.ChainerBackend'

# *** Manteniendo como lo tenías ***
yagmail.register('elpaniaguado@gmail.com', 'rnvg wpwb opim kwrp')

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, JavascriptException
import time

from webdriver_manager.chrome import ChromeDriverManager

# --- WebDriver ---
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.binary_location = '/usr/bin/chromium-browser'
# logs de navegador (por si quieres ver errores JS)
chrome_options.set_capability('goog:loggingPrefs', {'browser': 'ALL'})

# *** Manteniendo como lo tenías ***
yagmail.register('elpaniaguado@gmail.com', 'rnvg wpwb opim kwrp')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options=chrome_options)

def dump_debug(name="debug"):
    try:
        path_png = f"/tmp/{name}.png"
        path_html = f"/tmp/{name}.html"
        driver.save_screenshot(path_png)
        with open(path_html, "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        print(f"[DEBUG] Screenshot: {path_png}")
        print(f"[DEBUG] HTML: {path_html}")
        sys.stdout.flush()
    except Exception as e:
        print(f"[DEBUG] No se pudo guardar debug: {e}", flush=True)

# -------- Helpers UI --------
def esperar_tabla(timeout=20):
    print("[DBG] Esperando tabla en #div_padres…")
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//*[@id="div_padres"]//table[contains(@class,"table") and contains(@class,"table-bordered")]'
        ))
    )
    print("[DBG] Tabla detectada.")

def click_vista_diaria(timeout=10):
    print("[DBG] Click 'Vista diaria'…")
    btn = WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, '//a[contains(normalize-space(.), "Vista diaria")]'))
    )
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
    driver.execute_script("arguments[0].click();", btn)
    esperar_tabla()
    print("[DBG] Click 'Vista diaria' OK.")

def click_vista_semanal(timeout=10):
    print("[DBG] Click 'Vista semanal'…")
    btn = WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, '//a[contains(normalize-space(.), "Vista semanal")]'))
    )
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
    driver.execute_script("arguments[0].click();", btn)
    esperar_tabla()
    print("[DBG] Click 'Vista semanal' OK.")

def set_fecha_en_input_y_disparar_change(fecha_str: str):
    """
    Pone fecha en #fecha_incidencia y dispara el onchange para que recargue la vista diaria.
    Usa jQuery si está presente; si no, Event('change') nativo.
    """
    print(f"[DBG] Set #fecha_incidencia = {fecha_str} + change()…")
    js = """
        const f = arguments[0];
        const inp = document.getElementById('fecha_incidencia');
        if (!inp) return 'NO_INPUT';
        inp.value = f;
        if (window.jQuery) {
            jQuery(inp).trigger('change');
            return 'JQ_CHANGE';
        } else {
            const ev = new Event('change', {bubbles:true});
            inp.dispatchEvent(ev);
            return 'NATIVE_CHANGE';
        }
    """
    res = driver.execute_script(js, fecha_str)
    print(f"[DBG] Resultado set/trigger: {res}")

def esperar_header_de_fecha(fecha_str: str, timeout=25):
    print(f"[DBG] Esperando <th> con '{fecha_str}'…")
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((
            By.XPATH, f'//*[@id="div_padres"]//th[contains(normalize-space(.), "{fecha_str}")]'
        ))
    )
    print("[DBG] Header con la fecha detectado.")

def get_tabla_html() -> str:
    tabla_elem = driver.find_element(
        By.XPATH,
        '//*[@id="div_padres"]//table[contains(@class,"table") and contains(@class,"table-bordered")]'
    )
    html = tabla_elem.get_attribute("outerHTML")
    print(f"[DBG] Longitud HTML tabla: {len(html)}")
    return html

def cargar_vista_semanal_para(fecha_str: str) -> str:
    """
    Entra en vista semanal (botón) y devuelve la tabla semanal.
    Intenta además forzar por JS a la semana de esa fecha (opcional).
    """
    click_vista_semanal()
    try:
        print(f"[DBG] (opcional) Forzando semanal JS para {fecha_str}…")
        driver.execute_script(
            'return CargaDiv("div_padres","padre","padre_trabajo","fecha", arguments[0], "tipo", 1);',
            fecha_str
        )
        # no todas las semanales muestran la fecha exacta en un <th>; no bloqueamos si no aparece
        time.sleep(1.0)
    except Exception as e:
        print(f"[DBG] No forcé semanal por JS: {e}")
    return get_tabla_html()

def cargar_vista_diaria_para(fecha_str: str) -> str:
    """
    Entra en vista diaria, fija #fecha_incidencia a fecha_str, dispara change,
    espera el encabezado de esa fecha y devuelve la tabla HTML del día.
    """
    click_vista_diaria()
    # asegurar input visible
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'fecha_incidencia')))
    set_fecha_en_input_y_disparar_change(fecha_str)
    esperar_header_de_fecha(fecha_str, timeout=25)
    return get_tabla_html()

try:
    # --- Login ---
    driver.get('https://lasallesanrafael.sallenet.org')
    time.sleep(3)
    username_field = driver.find_element(By.ID, 'username')
    password_field = driver.find_element(By.ID, 'password')

    # *** Manteniendo como lo tenías ***
    username_field.send_keys('davidpaniaguadominguez')
    password_field.send_keys('Valhalla99!')

    driver.find_element(By.ID, 'loginbtn').click()
    time.sleep(10)
    print("Login exitoso")

    # --- Padres -> Trabajo ---
    driver.get('https://lasallesanrafael.sallenet.org/mod/sallenet/modulos/padres/')
    time.sleep(10)
    print("Navegación a 'padres' OK")

    enlace_trabajo = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'Trabajo')]"))
    )
    driver.execute_script("arguments[0].click();", enlace_trabajo)
    esperar_tabla()

    # --- Fecha hoy (Europe/Madrid) ---
    try:
        from zoneinfo import ZoneInfo
        hoy_str = datetime.now(ZoneInfo("Europe/Madrid")).strftime("%d/%m/%Y")
    except Exception:
        hoy_str = datetime.utcnow().strftime("%d/%m/%Y")
    print(f"[DBG] Hoy: {hoy_str}")

    # 1) Capturar SEMANA primero
    print("[DBG] Cargando VISTA SEMANAL…")
    tabla_semana_html = cargar_vista_semanal_para(hoy_str)
    print("[DBG] Semanal capturada ✅")

    # 2) Cambiar a DIARIA y capturar
    print("[DBG] Cargando VISTA DIARIA…")
    tabla_dia_html = cargar_vista_diaria_para(hoy_str)
    print("[DBG] Diaria capturada ✅")

    # --- Email ---
    estilos_css = """
    <style>
        table { width: 100%; border-collapse: collapse; }
        th, td { border: 1px solid #dddddd; text-align: left; padding: 8px; vertical-align: top; }
        th { background-color: #f2f2f2; }
        tr:nth-child(even) { background-color: #f9f9f9; }
        h1 { margin: 0 0 10px; }
    </style>
    """

    tabla_html_modificada = tabla_dia_html        # DÍA ACTUAL
    tabla_html_modificada2 = tabla_semana_html    # SEMANA COMPLETA

    yag = yagmail.SMTP('elpaniaguado@gmail.com')

    # *** Manteniendo tus listas ***
    destinatarios3 = [
        'elpaniaguado@gmail.com',
        'cquinteromoreno@gmail.com',
        'sharonrdc@yahoo.es',
        'abdjimenez@hotmail.com',
        'paradinasmarcos@gmail.com',
        'paulaherediaalcantara@gmail.com',
        'mariofvilladangos@hotmail.com',
        'adrian.alvarez.munoz@gmail.com',
        'nu.abril@gmail.com',
        'lorenitavelayos@gmail.com',
        'judiith22@gmail.com',
        'aransay13@gmail.com',
        'emmaedani@gmail.com',
        'anaisabelmas@hotmail.com',
        'juandiazlorenzo@hotmail.com'
    ]
    destinatarios = ['elpaniaguado@gmail.com']  # ajusta cuál usas

    contenido_html = f"""
    <html>
    <head>{estilos_css}</head>
    <body>
        <h1 style="font-weight:bold;">DIA ACTUAL</h1>
        {tabla_html_modificada}
        <h1 style="font-weight:bold;">SEMANA COMPLETA</h1>
        {tabla_html_modificada2}
    </body>
    </html>
    """

    contenido_html_inline = css_inline.inline(contenido_html)
    contenido_html_min = minify_html(contenido_html_inline)

    dia_actual_str = datetime.now().strftime("%d-%m-%Y")
    subject = "Calendario de tareas La Salle 1B " + dia_actual_str

    try:
        yag.send(
            to=destinatarios,  # o destinatarios3
            subject=subject,
            contents=[yagmail.raw(contenido_html_min)]
        )
        print("Correo enviado con éxito a todos los destinatarios.")
    except Exception as e:
        print(f"Error al enviar el correo: {e}")

except Exception as e:
    print("Fallo:", e)
    dump_debug("fatal")
    # logs de navegador por si hay error JS
    try:
        for entry in driver.get_log('browser'):
            print(f"[BROWSER] {entry.get('level')}: {entry.get('message')}")
    except Exception:
        pass
    raise
finally:
    driver.quit()
    print("Fin.")




Empiezo


ModuleNotFoundError: No module named 'yagmail'